# Prepare environment

In [1]:
import os
from google.colab import drive
drive.mount('/gdrive')
os.symlink('/gdrive/My Drive', '/content/gdrive')
!ls -l /content/gdrive/


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
total 28
drwx------ 2 root root 4096 Oct  3  2018 'Colab Notebooks'
drwx------ 2 root root 4096 Jun  2  2018  DataScience
drwx------ 2 root root 4096 Feb  6  2019  Notability
drwx------ 2 root root 4096 May 12 21:02 'Sleep as Android'
drwx------ 2 root root 4096 Apr 21 16:20 'Trekking '
drwx------ 2 root root 4096 Jun 14 20:30 'Wedding '
drwx------ 2 root root 4096 Sep 15  2018  WWW


In [2]:
!ls gdrive/DataScience/DATA/01_otodom_scrapy_prepred

array_photo.npz  train_data.feather


In [3]:
!ls gdrive/DataScience/Projects/201907_xFlat_AWS_Scrapy

event.json   lambda_functions.py  requirements.txt  template.yaml
hello_world  Notebooks		  scraper	    tests
helpers      README.md		  scrapy.cfg	    venv


In [0]:
%%capture
!pip install -r gdrive/DataScience/Projects/201907_xFlat_AWS_Scrapy/requirements.txt

# Import data

In [0]:
import sys
sys.path.append("gdrive/DataScience/Projects/201907_xFlat_AWS_Scrapy")
import helpers 
import feather 
import pandas as pd
import numpy as np
import os


In [0]:
path = "gdrive/DataScience/DATA/01_otodom_scrapy_prepred"

In [0]:
df = pd.read_feather(os.path.join(path,'train_data.feather'))

In [8]:
df.head()

,offer_id,tracking_id,name,location,flat_size,rooms,floor,price,price_m2,market,number_of_floors,floor_attic,floor_basement,building_type,building_material,widows_type,heating_type,year_of_building,finishing_stage,rent_price,property_form,available_from,description,additional_info,GC_latitude,GC_llongitude,GC_boundingbox,GC_addr_house_number,GC_addr_road,GC_addr_neighbourhood,GC_addr_suburb,GC_addr_city,GC_addr_county,GC_addr_state,GC_addr_postcode,GC_addr_country,GC_addr_country_code,url,main_url
0,42ab6,59621128,3 pokoje BEMOWO bezpośrednio,"Warszawa, Bemowo",61.60,3,9.0,540000.0,8766.0,wtórny,10.0,0,0,blok,wielka płyta,plastikowe,miejskie,1986.0,do zamieszkania,600.0,spółdzielcze wł. z KW,None,Sprzedam \nbezpośrednio\n trzypokojowe mieszka...,drzwi / okna antywłamaniowe|zmywarka|lodówka|m...,52.24091,20.91133,"52.2402751,52.2408848,20.9106149,20.911858",200A,None,Osiedle Górczewska,Bemowo,Warszawa,Warszawa,mazowieckie,01-460,RP,pl,https://www.otodom.pl/oferta/3-pokoje-bemowo-b...,otodom.pl
1,42b9d,59624855,2 oddzielne pok. przy Rynku Nowego Miasta,"Warszawa, Śródmieście, Nowe Miasto, Ciasna",50.35,2,0.0,599999.0,11917.0,wtórny,3.0,0,0,kamienica,cegła,plastikowe,miejskie,1955.0,do zamieszkania,530.0,pełna własność,None,"Ładne, jasne i ciche mieszkanie w rewelacyjnej...",telewizja kablowa|internet|piwnica|oddzielna k...,52.2523695,21.00495568,"52.2521502,52.2526425,21.0045833,21.0051848",15,Ciasna,II,Śródmieście,Warszawa,Warszawa,mazowieckie,00-232,RP,pl,https://www.otodom.pl/oferta/2-oddzielne-pok-p...,otodom.pl
2,410cK,59344462,Ostatni Lokal ! 2 pokojowy loft w kamienicy,"Warszawa, Targówek, Księcia Ziemowita",42.10,2,4.0,289000.0,6865.0,wtórny,4.0,0,0,kamienica,cegła,None,None,1935.0,None,NaN,None,None,Zakup bezpośrednio od dewelopera! Bez prowizji...,,52.2626939,21.06144616,"52.2626137,52.2627745,21.0613101,21.0615822",4A,Księcia Ziemowita,Targówek Fabryczny,Targówek,Warszawa,Warszawa,mazowieckie,03-778,RP,pl,https://www.otodom.pl/oferta/ostatni-lokal-2-p...,otodom.pl
3,42clg,59629446,"Apartament , 4 pokoje","Warszawa, Mokotów, Bielawska",139.90,4,1.0,1273000.0,9099.0,wtórny,4.0,0,0,blok,cegła,None,None,2000.0,do zamieszkania,NaN,None,None,mieszkanie na 1 piętrze w budynku położonym w ...,,52.19410135,21.02817888,"52.1937544,52.1944598,21.0274456,21.0282802",6,Bielawska,Henryków,Mokotów,Warszawa,Warszawa,mazowieckie,02-511,RP,pl,https://www.otodom.pl/oferta/apartament-4-poko...,otodom.pl
4,3U6cE,57699224,"Mieszkanie, 64,24 m², Warszawa","Warszawa, Białołęka, ul. Skarbka z Gór",64.24,3,2.0,433620.0,6750.0,pierwotny,3.0,0,0,None,inne,plastikowe,gazowe,2018.0,do wykończenia,NaN,None,2018-08-31,Skarbka Skwer to najnowszy projekt mieszkaniow...,domofon / wideofon|monitoring / ochrona|balkon...,52.3226232,21.0572195,"52.3224642,52.3226642,21.0571502,21.0573502",None,Skarbka z Gór,Grodzisk,Białołęka,Warszawa,Warszawa,mazowieckie,03-370,RP,pl,https://www.otodom.pl/oferta/mieszkanie-64-24-...,otodom.pl


In [9]:
with np.load(os.path.join(path,'array_photo.npz')) as data:
    print(data.files)
    photo_1 = data['a1']

['a1', 'a2', 'a3']


In [10]:
photo_1.shape

(15821, 224, 224, 3)

In [0]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras as ks

In [12]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)










87916544/87910968 [==============================] - 40s 0us/step


In [0]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
x = Dense(192, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(1)(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=ks.optimizers.Adam(lr=3e-3), loss='mean_squared_error')


In [38]:
model.fit(x=photo_1, y=np.array(df.price.fillna(df.price.mean()).to_list()).reshape(-1,1), batch_size=512, epochs=1, verbose=True)

Epoch 1/1
15821/15821 [==============================] - 39s 2ms/step - loss: 1072083103440.8323


In [39]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)


0 input_1
1 conv2d_1
2 batch_normalization_1
3 activation_1
4 conv2d_2
5 batch_normalization_2
6 activation_2
7 conv2d_3
8 batch_normalization_3
9 activation_3
10 max_pooling2d_1
11 conv2d_4
12 batch_normalization_4
13 activation_4
14 conv2d_5
15 batch_normalization_5
16 activation_5
17 max_pooling2d_2
18 conv2d_9
19 batch_normalization_9
20 activation_9
21 conv2d_7
22 conv2d_10
23 batch_normalization_7
24 batch_normalization_10
25 activation_7
26 activation_10
27 average_pooling2d_1
28 conv2d_6
29 conv2d_8
30 conv2d_11
31 conv2d_12
32 batch_normalization_6
33 batch_normalization_8
34 batch_normalization_11
35 batch_normalization_12
36 activation_6
37 activation_8
38 activation_11
39 activation_12
40 mixed0
41 conv2d_16
42 batch_normalization_16
43 activation_16
44 conv2d_14
45 conv2d_17
46 batch_normalization_14
47 batch_normalization_17
48 activation_14
49 activation_17
50 average_pooling2d_2
51 conv2d_13
52 conv2d_15
53 conv2d_18
54 conv2d_19
55 batch_normalization_13
56 batch_norma

In [0]:
# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

In [0]:
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
model.compile(optimizer=ks.optimizers.Adam(lr=3e-3), loss='mean_squared_error')


In [0]:
y = np.log(np.array(df.price.fillna(df.price.mean()).to_list()).reshape(-1,1))

In [85]:
model.fit(x=photo_1, y=y, batch_size=512, epochs=5, verbose=True)

Epoch 1/5
15821/15821 [==============================] - 44s 3ms/step - loss: nan
Epoch 2/5
 1024/15821 [>.............................] - ETA: 40s - loss: nan

KeyboardInterrupt: ignored

In [0]:
from sklearn.metrics import r2_score

In [44]:
y_pred = np.exp(model.predict(x=photo_1, batch_size=512, verbose=True))

15821/15821 [==============================] - 40s 3ms/step


In [0]:
y_try = np.log(np.array(df.price.fillna(df.price.mean()).to_list()).reshape(-1,1))

In [49]:
r2_score(y,y_pred)

-11.23421340679138

In [72]:
y_pred.shape

(15821, 1)

In [51]:
y

array([[ 540000.],
       [ 599999.],
       [ 289000.],
       ...,
       [ 599000.],
       [ 569000.],
       [3500000.]])

In [50]:
y_pred

array([[2407794.5],
       [3846095.5],
       [3684581.8],
       ...,
       [3539564.8],
       [3468842.5],
       [1350487.8]], dtype=float32)